In [16]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import statistics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection

In [19]:
sms_2 = pd.read_csv('sms_spam.csv', encoding="ISO-8859-1")
sms_2 = sms_2.loc[:, ~sms_2.columns.str.contains('^Unnamed')]
print(sms_2.head(5))

     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [20]:
sms_2.columns = ['label', 'message']
print(sms_2.head(5))

  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [22]:
import string
from nltk.corpus import stopwords

def text_process(mess):
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # check dau cau
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join lai
    nopunc = ''.join(nopunc)
    #Detele stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [24]:
import string, nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean_msg_2(mess):
    mess = mess.lower()
    
    #remove “unnecessary” words: email address, phone number, etc
    pattern = '((978[\--– ])?[0-9][0-9\--– ]{10}[\--– ][0-9xX])|((978)?[0-9]{9}[0-9Xx])'
    mess = mess.sub(pattern, '', mess,0)
    
    # remove khoang trang
    mess = " ".join(mess.split())
    
    # lemmatize string: go = goes
    word_tokens = word_tokenize(mess)
    mess = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    # delete stopwỏd
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

for i in sms_2['message']:
    sms_2 = sms_2.replace([i],text_process(i))

print(sms_2['message'])

0       Go jurong point crazy Available bugis n great ...
1                                   Ok lar Joking wif oni
2       Free entry wkly comp win FA Cup final tkts 21s...
3                         dun say early hor c already say
4                  Nah think goes usf lives around though
                              ...                        
5567    2nd time tried contact å£750 Pound prize claim...
5568                          Ì b going esplanade fr home
5569                          Pity mood Soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       Rofl true name
Name: message, Length: 5572, dtype: object


In [26]:
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn import metrics
import time

y = sms.label
model = tree.DecisionTreeClassifier()

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
import time
start = time.time()

tfidf = TfidfVectorizer(min_df=5,max_df= 0.8,max_features=3000,sublinear_tf=True)
tfidf.fit(sms['message'])
X_Vectorizer = tfidf.transform(sms['message'])

# prepare the cross-validation procedure
cv = KFold(n_splits=5, random_state=1, shuffle=True)

# evaluate model
scores_1 = cross_val_score(model, X_Vectorizer, y, scoring='accuracy', cv=cv, n_jobs=-1)

end = time.time()
print("Time:",end - start)
print("Mean of Scores: ",mean(scores_1))
print("Scores:", scores_1)

Time: 2.3052163124084473
Mean of Scores:  0.9694904638075533
Scores: [0.97578475 0.96233184 0.96678636 0.96858169 0.97396768]


In [30]:
model = DecisionTreeRegressor()

cv = KFold(n_splits=10, random_state=1, shuffle=True)

scores = cross_val_score(model, X_Vectorizer, y, scoring='accuracy', cv=cv, n_jobs=-1)

print("Mean of accuracy: ",scores)

Mean of accuracy:  [nan nan nan nan nan nan nan nan nan nan]


In [31]:
cart = DecisionTreeRegressor()

model = BaggingRegressor(base_estimator=cart,n_estimators=num_trees, random_state=rng)

results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

NameError: name 'num_trees' is not defined